<a href="https://colab.research.google.com/github/FifthElement5/cwiczenia/blob/master/kod_monitora_kurs_w_walut_z_komentarzami.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests # Importuje bibliotekę do wykonywania zapytań HTTP (do pobierania danych z API).
import time # Importuje bibliotekę do operacji związanych z czasem (np. do pauzowania programu).

# Abstrakcyjna klasa bazowa dla dostawców kursów walut.
# Definiuje interfejs, który muszą implementować wszyscy dostawcy.
class RateProvider:
    def get_rate(self) -> float:
        # Abstrakcyjna metoda, która musi zostać zaimplementowana przez podklasy.
        # Powinna zwracać aktualny kurs waluty jako liczbę zmiennoprzecinkową.
        raise NotImplementedError

# Adapter dla API NBP, implementujący RateProvider.
# Odpowiada za pobieranie kursu EUR z serwisu NBP.
class NBPApiAdapter(RateProvider):
    def get_rate(self):
        # URL do API NBP dla kursu EUR (tabela A, format JSON).
        url = "https://api.nbp.pl/api/exchangerates/rates/A/EUR/?format=json"
        try:
            # Wykonuje zapytanie GET do API NBP.
            response = requests.get(url)
            # Parsuje odpowiedź JSON.
            data = response.json()
            # Zwraca średni kurs (mid) z pierwszej pozycji w liście kursów.
            return data["rates"][0]["mid"]
        except Exception as e:
            # Obsługuje błędy, które mogą wystąpić podczas pobierania lub parsowania danych.
            print("Błąd pobierania z NBP API:", e)
            return None # Zwraca None w przypadku błędu.

# Abstrakcyjna klasa bazowa dla obserwatorów kursów walut.
# Definiuje interfejs dla obiektów, które chcą być powiadamiane o zmianach kursu.
class RateObserver:
    def update(self, rate: float, provider: str):
        # Abstrakcyjna metoda, która musi zostać zaimplementowana przez podklasy.
        # Jest wywoływana, gdy kurs ulegnie zmianie.
        # 'rate' to aktualny kurs, 'provider' to nazwa dostawcy kursu.
        raise NotImplementedError

# Obserwator, który symuluje "inteligentnego kupującego".
# Śledzi kursy i "kupuje" po najkorzystniejszej cenie.
class SmartBuyer(RateObserver):
    def __init__(self):
        # Inicjalizuje najlepszy znaleziony kurs na nieskończoność,
        # aby każdy pierwszy kurs był od razu lepszy.
        self.best_rate = float('inf')

    def update(self, rate, provider):
        # Jeśli kurs jest None (np. z powodu błędu pobierania), ignoruje go.
        if rate is None:
            return
        # Wyświetla aktualny kurs od danego dostawcy.
        print(f"Kurs z {provider}: {rate}")
        # Sprawdza, czy aktualny kurs jest niższy niż dotychczasowy najlepszy kurs.
        if rate < self.best_rate:
            self.best_rate = rate # Aktualizuje najlepszy kurs.
            # Symuluje "zakup" po nowym, lepszym kursie.
            print(f"Kupuję EUR po kursie {rate} z {provider}!")

# Klasa monitorująca kursy walut.
# Jest "podmiotem" (Subject) we wzorcu Obserwator.
class RateMonitor:
    def __init__(self):
        # Lista zarejestrowanych dostawców kursów.
        self.providers = []
        # Lista zarejestrowanych obserwatorów kursów.
        self.observers = []

    def add_provider(self, provider: RateProvider):
        # Dodaje nowego dostawcę kursów do listy.
        self.providers.append(provider)

    def add_observer(self, observer: RateObserver):
        # Dodaje nowego obserwatora do listy.
        self.observers.append(observer)

    def check_rates(self):
        # Iteruje przez wszystkich zarejestrowanych dostawców.
        for p in self.providers:
            # Pobiera kurs od bieżącego dostawcy.
            rate = p.get_rate()
            # Iteruje przez wszystkich zarejestrowanych obserwatorów.
            for o in self.observers:
                # Powiadamia każdego obserwatora o aktualnym kursie i nazwie dostawcy.
                o.update(rate, p.__class__.__name__)

# --- Główna część programu ---

# Tworzy instancję monitora kursów.
monitor = RateMonitor()
# Dodaje adapter API NBP jako dostawcę kursów.
monitor.add_provider(NBPApiAdapter())
# Dodaje inteligentnego kupującego jako obserwatora.
monitor.add_observer(SmartBuyer())

try:
    # Nieskończona pętla do ciągłego monitorowania kursów.
    while True:
        print("\n--- Sprawdzam kursy ---")
        # Wywołuje metodę sprawdzającą kursy i powiadamiającą obserwatorów.
        monitor.check_rates()
        # Pauzuje program na 30 sekund przed kolejnym sprawdzeniem.
        time.sleep(30)
except KeyboardInterrupt:
    # Obsługuje przerwanie programu przez użytkownika (np. Ctrl+C).
    print("\nZamykanie monitora kursów...")